Import Libraries


In [6]:
import numpy as np
from osgeo import gdal
from scipy import stats
import timeit

<img src="data/thumb.png" alt="Drawing" style="width: 300px;"/>

Load test data

In [7]:
r = gdal.Open("data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif")
g = np.array(r.GetRasterBand(1).ReadAsArray())

In [8]:
##stats.describe(g) 
print( (r.RasterYSize,r.RasterXSize))
print( g.shape )
##out.shape

(1734, 2501)
(1734, 2501)


Function to create data structure function takes in a grid with mxn and returns a grid with mxnxc where c is the number of classes.  A vectorized version would be faster

In [9]:
def dtFZS(g,out):
    for xw in range(0,g.shape[1]):
        for yw in range(1,g.shape[0]):
            out[yw,xw] = out[yw-1,xw] + g[yw,xw]
    print("done")

In [10]:
##out = dtFZS(g)
DS_FstZS = np.zeros(g.shape,dtype='uint64')
dtFZS(g,DS_FstZS)


done


In [11]:
stats.describe(DS_FstZS)
##out.shape

DescribeResult(nobs=1734, minmax=(array([0, 0, 0, ..., 0, 0, 0], dtype=uint64), array([ 88957,  90090,  90537, ..., 113531, 112942, 113583], dtype=uint64)), mean=array([ 39105.28258362,  39471.85870819,  39549.93886967, ...,
        55162.80046136,  54622.01153403,  54913.44925029]), variance=array([  7.33292121e+08,   7.43734955e+08,   7.51916039e+08, ...,
         1.12397767e+09,   1.11742125e+09,   1.11700624e+09]), skewness=array([ 0.20808248,  0.21883235,  0.21780447, ...,  0.09384659,
        0.09849129,  0.1000836 ]), kurtosis=array([-1.25728163, -1.23819352, -1.22937554, ..., -1.22189322,
       -1.22273959, -1.20557672]))

Fast Summary Statistics Function
    

In [12]:
##  To DO
## This function needs to be rewritten to make v a lattice on the grid (not matching the grid locations but between the grid valeus)

def FstZS(v,DS_FstZS):
    total = 0;
    for p in range(1,len(v)):
        ## If moving left
        if v[p][1] - v[p-1][1] > 0:
            total += DS_FstZS[v[p][0],v[p][1]]
        ## If moving Right
        elif v[p][1] - v[p-1][1] < 0:
            total -= DS_FstZS[v[p][0],v[p][1]]
    return int(total)

In [17]:
## Test Data set
z = 1000
## Create test box z in size
v = [(x,1) for x in range(1,z+1)] + [(z,x) for x in range(1+1,z+1)] + [(x,z) for x in range(z-1,0,-1)] + [(1,x) for x in range(z-1,0,-1)]
v

[(1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 1),
 (77, 1),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 1),
 (99, 1),
 (100, 1),
 (101, 1

In [14]:
#g.shape
FstZS(v,DS_FstZS)
#%timeit 'FstZS(v,DS_FstZS)'

47168172

In [15]:
##%timeit 'np.sum(g[1:1001,1:1001])'
np.sum(g[2:1001,2:1001])

47168226

Pretty Close Not sure why these are slightly different end